# Using EcoFOCIpy to process raw field data

## NW2301 (BASIS - NWE)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2023/CTDcasts/nw2301/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'NW2301' #no hyphens
cruise_meta_file = sample_data_dir+'logs/NW2301.yaml'
inst_meta_file = sample_data_dir+'logs/EMA_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD001.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD002.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD003.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD004.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD005.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD006.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD007.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD008.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD009.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD010.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD011.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD012.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD013.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD021.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD022._bottle_refirehex.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD022.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD023hex.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD024.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD025.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD026.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD027.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD028.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD029.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD030.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD031.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CT

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD044.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD045.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/nw2301/rawconverted/CTD046.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['CTD001.btl'].describe()

,sal11,sal00,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sigma-t00,sigma-t11,c0ms/cm,c1ms/cm,fleco-afl,t090c,t190c,scan,prdm,datetime
count,3.0000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3
mean,30.9017,30.90290,6.618800,99.553667,288.686667,6.497033,97.721667,283.374000,23.933267,23.932267,33.091567,33.090466,1.787767,8.857133,8.857200,4780.000000,7.014000,2023-08-30 08:04:11.333333248
min,30.9013,30.90260,6.441500,96.889000,280.952000,6.477200,97.419000,282.508000,23.933100,23.932100,33.087634,33.086139,1.697400,8.855200,8.855000,4367.000000,2.082000,2023-08-30 08:03:54
25%,30.9015,30.90275,6.461600,97.191500,281.830500,6.491150,97.632000,283.116500,23.933200,23.932200,33.090538,33.089369,1.742300,8.856600,8.856650,4449.500000,5.604500,2023-08-30 08:03:57.500000
50%,30.9017,30.90290,6.481700,97.494000,282.709000,6.505100,97.845000,283.725000,23.933300,23.932300,33.093442,33.092598,1.787200,8.858000,8.858300,4532.000000,9.127000,2023-08-30 08:04:01
75%,30.9019,30.90305,6.707450,100.886000,292.554000,6.506950,97.873000,283.807000,23.933350,23.932350,33.093534,33.092630,1.832950,8.858100,8.858300,4986.500000,9.480000,2023-08-30 08:04:20
max,30.9021,30.90320,6.933200,104.278000,302.399000,6.508800,97.901000,283.889000,23.933400,23.932400,33.093625,33.092661,1.878700,8.858200,8.858300,5441.000000,9.833000,2023-08-30 08:04:39
std,0.0004,0.00030,0.273019,4.102560,11.907679,0.017276,0.263608,0.754447,0.000153,0.000153,0.003407,0.003747,0.090651,0.001677,0.001905,578.357156,4.285799,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'NW2301',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NorthWest Explorer',
 'ShipID': 'NW',
 'StartDate': datetime.date(2023, 8, 30),
 'EndDate': datetime.date(2023, 9, 15),
 'Project': 'NBS',
 'ChiefScientist': 'Dan Cooper',
 'StartPort': '',
 'EndPort': '',
 'CruiseLocation': 'Bering',
 'Description': 'NBS',
 'CruiseYear': 2023,
 'ctdlogs_pdf_name': ''}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54374,
 'Vessel': 'NorthWest Explorer',
 'CruiseID': 'NW2301',
 'Project_Leg': '',
 'UniqueCruiseID': 'NW2301',
 'Project': 'BASIS',
 'StationNo_altname': '1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 60,
 'LatitudeMin': 0.138,
 'LongitudeDeg': 168,
 'LongitudeMin': 0.72,
 'GMTDay': 30,
 'GMTMonth': 'Aug',
 'GMTYear': 2023,
 'GMTTime': 57647,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': '',
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 21,
 'StationNameID': '',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': 'shallow station, wire speed seemed fast\r',
 'NutrientBtlNiskinNo': '1;2;3',
 'NutrientBtlNumbers': '1;4',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'Wa

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['CTD001.btl'] = cruise_data['CTD001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['CTD001.btl'].sample()

,salinity_ch2,salinity_ch1,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,sigma_t_ch1,sigma_t_ch2,c0ms/cm,c1ms/cm,chlor_fluorescence,temperature_ch1,temperature_ch2,scan,Pressure [dbar],datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
3.0,30.9017,30.9029,6.4415,96.889,280.952,6.5051,97.845,283.725,23.9331,23.9321,33.093625,33.092598,1.6974,8.8582,8.8583,4367.0,9.833,2023-08-30 08:03:54,ctd001


In [10]:
cruise_data['CTD001.btl'].columns

Index(['salinity_ch2', 'salinity_ch1', 'oxy_concM_ch1', 'oxy_percentsat_ch1',
       'oxy_conc_ch1', 'oxy_concM_ch2', 'oxy_percentsat_ch2', 'oxy_conc_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'c0ms/cm', 'c1ms/cm',
       'chlor_fluorescence', 'temperature_ch1', 'temperature_ch2', 'scan',
       'Pressure [dbar]', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['CTD001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 480B
Dimensions:             (bottle: 3)
Coordinates:
  * bottle              (bottle) float64 24B 3.0 4.0 5.0
Data variables: (12/19)
    salinity_ch2        (bottle) float64 24B 30.9 30.9 30.9
    salinity_ch1        (bottle) float64 24B 30.9 30.9 30.9
    oxy_concM_ch1       (bottle) float64 24B 6.441 6.482 6.933
    oxy_percentsat_ch1  (bottle) float64 24B 96.89 97.49 104.3
    oxy_conc_ch1        (bottle) float64 24B 281.0 282.7 302.4
    oxy_concM_ch2       (bottle) float64 24B 6.505 6.509 6.477
    ...                  ...
    temperature_ch1     (bottle) float64 24B 8.858 8.858 8.855
    temperature_ch2     (bottle) float64 24B 8.858 8.858 8.855
    scan                (bottle) float64 24B 4.367e+03 4.532e+03 5.441e+03
    Pressure [dbar]     (bottle) float64 24B 9.833 9.127 2.082
    datetime            (bottle) datetime64[ns] 24B 2023-08-30T08:03:54 ... 2...
    cast                (bottle) object 24B 'ctd001' 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['CTD001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 504B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 3)
Coordinates:
  * bottle              (bottle) float64 24B 3.0 4.0 5.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/19)
    salinity_ch2        (latitude, longitude, time, bottle) float64 24B 30.9 ...
    salinity_ch1        (latitude, longitude, time, bottle) float64 24B 30.9 ...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 24B 6.441...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 24B 96.89...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 24B 281.0...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 24B 6.505...
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 24B 8.858...
    temperature_ch2     (latitude, longitude, time, bottle) float64 24B 8.858...
    scan                (latitude, longitude, time, bottle) float64 24B 4.367...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 24B 9.833...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 24B ...
    cast                (latitude, longitude, time, bottle) object 24B 'ctd00...

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 504B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 3)
Coordinates:
  * bottle              (bottle) float64 24B 3.0 4.0 5.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/19)
    salinity_ch2        (latitude, longitude, time, bottle) float64 24B 30.9 ...
    salinity_ch1        (latitude, longitude, time, bottle) float64 24B 30.9 ...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 24B 6.441...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 24B 96.89...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 24B 281.0...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 24B 6.505...
    ...                  ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 24B 8.858...
    temperature_ch2     (latitude, longitude, time, bottle) float64 24B 8.858...
    scan                (latitude, longitude, time, bottle) float64 24B 4.367...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 24B 9.833...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 24B ...
    cast                (latitude, longitude, time, bottle) object 24B 'ctd00...
Attributes: (12/44)
    CruiseID:                      NW2301
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NorthWest Explorer
    ShipID:                        NW
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-10-17T10:34:29Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'par/sat/log':'par',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

Skipping CTD023hex.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history